In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 1. Load and Preprocess Data
# Assume you have a CSV file with columns: 'Artist', 'Song Title', 'Release Year', 'Genre', 'Lyrics', 'Topic', 'Interested'
# Load dataset
df = pd.read_csv("lyrics_data_music.csv")

# Drop rows with any missing values in the important columns
df = df.dropna(subset=['Artist', 'Song Title', 'Release Year', 'Genre', 'Lyrics', 'Topic'])

# Remove stopwords from lyrics
df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df['Interested'] = df['Genre'].apply(lambda x: 'Yes' if x in ['pop', 'blues'] else 'No')
# 2. Split Data
X = df['Lyrics']
y = df['Interested']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0.0, max_df=1.0, max_features=1000, lowercase=True)
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)


X_train = x_train.astype('float32')
y_train = y_train.replace({'No': 0, 'Yes': 1}).astype('float32')
X_test = x_test.astype('float32')
y_test = y_test.replace({'No': 0, 'Yes': 1}).astype('float32')

from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saite\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Latent Semantic Analysis (LSA) for Topic Reduction with Logistic regression

In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression

# Dimensionality reduction with LSA
lsa = TruncatedSVD(n_components=50, random_state=42)
X_train_reduced = lsa.fit_transform(X_train)
X_test_reduced = lsa.transform(X_test)

# Train a Logistic Regression model
logreg = LogisticRegression()
logreg.fit(X_train_reduced, y_train)

# Evaluate the model
y_pred = logreg.predict(X_test_reduced)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.598590308370044
              precision    recall  f1-score   support

         0.0       0.61      0.91      0.73      3351
         1.0       0.54      0.15      0.23      2324

    accuracy                           0.60      5675
   macro avg       0.57      0.53      0.48      5675
weighted avg       0.58      0.60      0.53      5675



In [3]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


### Gradient Boosting for Classification

In [4]:
from xgboost import XGBClassifier

# Train XGBoost Classifier
xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)

# Predict and evaluate
y_pred = xgb.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.6052863436123348
              precision    recall  f1-score   support

         0.0       0.63      0.82      0.71      3351
         1.0       0.53      0.29      0.38      2324

    accuracy                           0.61      5675
   macro avg       0.58      0.56      0.54      5675
weighted avg       0.59      0.61      0.57      5675



### LSTM

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# 1. Load and Preprocess Data
# Assume you have a CSV file with columns: 'Artist', 'Song Title', 'Release Year', 'Genre', 'Lyrics', 'Topic', 'Interested'
df = pd.read_csv("lyrics_data_music.csv")

# Drop rows with any missing values in the important columns
df = df.dropna(subset=['Artist', 'Song Title', 'Release Year', 'Genre', 'Lyrics', 'Topic'])

# Remove stopwords from lyrics
df['Lyrics'] = df['Lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
df['Interested'] = df['Genre'].apply(lambda x: 'Yes' if x in ['pop', 'blues'] else 'No')

# Split Data
X = df['Lyrics']
y = df['Interested'].replace({'No': 0, 'Yes': 1})  # Convert labels to binary
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Tokenization and Padding for LSTM
tokenizer = Tokenizer(num_words=5000)  # Consider top 5000 words
tokenizer.fit_on_texts(x_train)

x_train_seq = pad_sequences(tokenizer.texts_to_sequences(x_train), maxlen=100)
x_test_seq = pad_sequences(tokenizer.texts_to_sequences(x_test), maxlen=100)

# 3. Build and Compile LSTM Model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),  # Embedding layer
    LSTM(128, return_sequences=False),  # LSTM layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Train the Model
model.fit(x_train_seq, y_train, epochs=5, batch_size=32, validation_data=(x_test_seq, y_test))

# 5. Evaluate the Model
loss, accuracy = model.evaluate(x_test_seq, y_test)
print("LSTM Accuracy:", accuracy)

# Optional: Classification Report
y_pred = (model.predict(x_test_seq) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saite\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\saite\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 254s 206ms/step - accuracy: 0.5956 - loss: 0.6726 - val_accuracy: 0.5908 - val_loss: 0.6665
Epoch 2/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 127s 176ms/step - accuracy: 0.6381 - loss: 0.6308 - val_accuracy: 0.6007 - val_loss: 0.6770
Epoch 3/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 118s 165ms/step - accuracy: 0.6920 - loss: 0.5750 - val_accuracy: 0.5894 - val_loss: 0.7008
Epoch 4/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 115s 161ms/step - accuracy: 0.7345 - loss: 0.5101 - val_accuracy: 0.5831 - val_loss: 0.7485
Epoch 5/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 102s 143ms/step - accuracy: 0.7675 - loss: 0.4484 - val_accuracy: 0.5655 - val_loss: 0.8665
178/178 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.5596 - loss: 0.8853
LSTM Accuracy: 0.5654625296592712
178/178 ━━━━━━━━━━━━━━━━━━━━ 8s 42ms/step
              precision    recall  f1-score   support

           0       0.63      0.64      0.63      3351
           1       0.47      0.46      0.46      2324

    accuracy                 

### Autoencoders for Feature Extraction

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the Autoencoder
autoencoder = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),  # Bottleneck layer
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dense(X_train.shape[1], activation='sigmoid')
])

autoencoder.compile(optimizer='adam', loss='mse')

# Train the Autoencoder
autoencoder.fit(X_train.toarray(), X_train.toarray(), epochs=5, batch_size=32, validation_data=(X_test.toarray(), X_test.toarray()))

# Extract features from the bottleneck layer
encoder = Sequential(autoencoder.layers[:3])
X_train_encoded = encoder.predict(X_train.toarray())
X_test_encoded = encoder.predict(X_test.toarray())

# Train a classifier on the encoded features
logreg = LogisticRegression()
logreg.fit(X_train_encoded, y_train)
y_pred = logreg.predict(X_test_encoded)
print("Accuracy:", accuracy_score(y_test, y_pred))

C:\Users\saite\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 19s 18ms/step - loss: 1.0241 - val_loss: 1.1113
Epoch 2/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - loss: 1.0056 - val_loss: 1.1122
Epoch 3/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 1.0149 - val_loss: 1.1128
Epoch 4/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 1.0186 - val_loss: 1.1133
Epoch 5/5
710/710 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 1.0007 - val_loss: 1.0852
710/710 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Accuracy: 0.5973568281938326


C:\Users\saite\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
